In [ ]:
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'

In [ ]:
import cv2
import numpy as np

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=320, height=240)

In [ ]:
image=camera.read()
print(image.shape)

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

In [ ]:
image_widget = ipywidgets.Image(value=bgr8_to_jpeg(image),format='jpeg',width=640, height=480)
display(image_widget)

In [ ]:
myColors = [[5,107,0,19,255,255],
            [133,56,0,159,156,255],
            [57,76,0,100,255,255],
            [90,48,0,118,255,255]]
myColorValues = [[51,153,255],          ## BGR
                 [255,0,255],
                 [0,255,0],
                 [255,0,0]]

myPoints =  []  ## [x , y , colorId ]


In [ ]:
def findColor(img,myColors,myColorValues):
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    count = 0
    newPoints=[]
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imgHSV,lower,upper)
        x,y=getContours(mask)
        cv2.circle(imgResult,(x,y),15,myColorValues[count],cv2.FILLED)
        if x!=0 and y!=0:
            newPoints.append([x,y,count])
        count +=1
        #cv2.imshow(str(color[0]),mask)
    return newPoints

In [ ]:
def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
            #cv2.drawContours(imgResult, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx)
    return x+w//2,y

In [ ]:
def drawOnCanvas(myPoints,myColorValues):
    for point in myPoints:
        cv2.circle(imgResult, (point[0], point[1]), 10, myColorValues[point[2]], cv2.FILLED)

In [ ]:
camera.running = True

def update_image(change):
    img = change['new']
    global imgResult
    imgResult = img.copy()
    newPoints = findColor(img, myColors,myColorValues)
    if len(newPoints)!=0:
        for newP in newPoints:
            myPoints.append(newP)
    if len(myPoints)!=0:
        drawOnCanvas(myPoints,myColorValues)
    image_widget.value = bgr8_to_jpeg(imgResult)

    
    
camera.observe(update_image, names='value')

In [ ]:
# camera.unobserve(update_image, names='value')